# Import libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip3 install pydicom
!pip3 install segmentation_models

In [ ]:

import warnings
warnings.filterwarnings('ignore')

import os
import cv2
import csv
import pickle
import pydicom
import numpy as np
import pandas as pd 
from glob import glob

os.chdir('/content/drive/MyDrive/Initial-Code')
#os.chdir('./Initial-Code')

# import the necessary packages
import keras
import tensorflow as tf
from keras import backend as K

from dataset import prepare_data
from metric_loss import my_iou_metric, iou_metric_batch_val, bce_dice_loss
%env SM_FRAMEWORK=tf.keras

from predict import predict_result_val, prepare_test, get_test, get_prediction, get_rles
from generator import DataGenerator, label_generator

import seg_models
keras.backend.set_image_data_format('channels_last')

from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint

import sys
sys.path.insert(0, 'siim-acr-pneumothorax-segmentation')
from mask_functions import rle2mask, mask2rle
#X
## Seeding 
seed = 1994
np.random.seed = seed
os.environ['PYTHONHASHSEED'] = str(seed)
tf.seed = seed

import gc   #Gabage collector for cleaning deleted data from memory

In [ ]:
#!pip3 install pydicom
#!pip3 install keras
#!pip3 install tensorflow
#!pip3 install sklearn
#!pip3 install segmentation_models
#!pip3 install generic_utils

#!pip3 install  albumentations
!pip3 install backbone-network

# Dataset

In [ ]:
# defining configuration parameters
org_size = 1024 # original image size
img_size = 256 # image resize size
batch_size = 10 # batch size for training unet

## Load train and validation data from files

In [ ]:
pkl_file_train = open('process_data/X_train.pkl', 'rb')

X_train = pickle.load(pkl_file_train)

In [ ]:
pkl_file_val = open('process_data/X_val.pkl', 'rb')

X_val = pickle.load(pkl_file_val)

In [ ]:
pkl_file_masks = open('process_data/masks.pkl', 'rb')

masks = pickle.load(pkl_file_masks)

## Data generation & Augmentations

In [ ]:
import albumentations as A

In [ ]:
training_augmentation = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.OneOf([
        #A.CLAHE(),
        A.RandomContrast(),
        A.RandomGamma(),
        A.RandomBrightness(),
         ], p=0.3),
    A.OneOf([
        A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
        A.GridDistortion(),
        A.OpticalDistortion(distort_limit=2, shift_limit=0.5),
        ], p=0.3),
    A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=20,
                                        interpolation=cv2.INTER_LINEAR, border_mode=cv2.BORDER_CONSTANT, p=1),
    A.RandomSizedCrop(min_max_height=(206,256), height=img_size, width=img_size,p=0.25)
],p=1)

In [ ]:
params_train = {'img_size': img_size,
          'batch_size': batch_size,
          'n_channels': 3,
          'shuffle': True,
           'augmentations':training_augmentation,
           }

params_val = {'img_size': img_size,
          'batch_size': batch_size,
          'n_channels': 3,
          'shuffle': True,
         }

# Generators
training_generator = DataGenerator(X_train, masks, **params_train)
validation_generator = DataGenerator(X_val, masks, **params_val)

In [ ]:
x, y = training_generator.__getitem__(0)
print(x.shape, y.shape)

# Segmentation model

In [ ]:
K.clear_session()

In [ ]:
BACKBONE = 'resnet50'
model = seg_models.Unet(backbone_name=BACKBONE, encoder_weights='imagenet') #, decoder_use_batchnorm=False)
model.summary()

In [ ]:
# From: https://github.com/jocicmarko/ultrasound-nerve-segmentation/blob/master/train.py
def dice_coef(y_true, y_pred):
    y_true_f = tf.keras.layers.flatten(y_true)
    y_pred_f = tf.keras.layers.flatten(y_pred)
    intersection = keras.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (keras.sum(y_true_f) + keras.sum(y_pred_f) + 1)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def unet(input_size=(256,256,1)):
    
    inputs = Input(input_size)
    
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    return Model(inputs=[inputs], outputs=[conv10])



In [ ]:
opt = SGD(momentum=0.9)

In [ ]:
model.compile(optimizer=opt, loss=bce_dice_loss, metrics=[my_iou_metric])

In [ ]:
from swa import SWA
from cosine_schedule import CosineAnnealingScheduler

In [ ]:
epochs = 60
swa = SWA('model_output/512_resnet50_swa.model',55)

callbacks = [
    ModelCheckpoint("model_output/512_resnet50.model",monitor='val_loss', 
                            mode = 'min', save_best_only=True,
                            verbose=1),
    swa,
    CosineAnnealingScheduler(T_max=epochs, eta_max=1e-3, eta_min=1e-5, verbose=1)
]

In [ ]:
history = model.fit_generator(generator=training_generator,
                            validation_data=validation_generator,   
                           epochs=epochs, verbose=1,
                            callbacks=callbacks)

In [ ]:
# list all data in history
import matplotlib.pyplot as plt
 
print(history.history.keys())

# summarize history for iou
plt.figure(figsize=(20,5))
plt.subplot(1,2,1)
plt.plot(history.history['my_iou_metric'])
plt.plot(history.history['val_my_iou_metric'])
plt.title('model IOU')
plt.ylabel('iou')
plt.xlabel('epoch')
plt.legend(['train', 'Validation'], loc='upper left')

# summarize history for loss
plt.subplot(1,2,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'Validation'], loc='upper left')

In [ ]:
# Load best model or swa model

model.load_weights('model_output/512_resnet50_swa.model')

#print('using best weight model')
#model.load_weights('stage2_model_output/256_resnet34.model')

In [ ]:
# defining configuration parameters
org_size = 1024 # original image size
img_size = 512# image resize size
batch_size = 10 # batch size for training unet

In [ ]:
pkl_file_val = open('process_data/X_val.pkl', 'rb')

X_val = pickle.load(pkl_file_val)

In [ ]:
pkl_file_val = open('process_data/X_val.pkl', 'rb')

X_val = pickle.load(pkl_file_val)

In [ ]:
pkl_file_masks = open('process_data/masks.pkl', 'rb')

masks = pickle.load(pkl_file_masks)

In [ ]:
import albumentations as A

In [ ]:
training_augmentation = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.OneOf([
        #A.CLAHE(),
        A.RandomContrast(),
        A.RandomGamma(),
        A.RandomBrightness(),
         ], p=0.3),
    A.OneOf([
        A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
        A.GridDistortion(),
        A.OpticalDistortion(distort_limit=2, shift_limit=0.5),
        ], p=0.3),
    A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=20,
                                        interpolation=cv2.INTER_LINEAR, border_mode=cv2.BORDER_CONSTANT, p=1),
    A.RandomSizedCrop(min_max_height=(412, 512), height=img_size, width=img_size,p=0.25)
],p=1)

In [ ]:
params_train = {'img_size': img_size,
          'batch_size': batch_size,
          'n_channels': 3,
          'shuffle': True,
           'augmentations':training_augmentation,
           }

params_val = {'img_size': img_size,
          'batch_size': batch_size,
          'n_channels': 3,
          'shuffle': True,
         }

# Generators
training_generator = DataGenerator(X_train, masks, **params_train)
validation_generator = DataGenerator(X_val, masks, **params_val)

In [ ]:
x, y = training_generator.__getitem__(0)
print(x.shape, y.shape)

In [ ]:
from swa import SWA
from cosine_schedule import CosineAnnealingScheduler

In [ ]:
epochs = 60
swa = SWA('model_output/512_resnet50_swa_stage2.model',55)

callbacks = [
    ModelCheckpoint("model_output/512_resnet50_stage2.model",monitor='val_loss', 
                            mode = 'min', save_best_only=True,
                            verbose=1),
    swa,
    CosineAnnealingScheduler(T_max=epochs, eta_max=1e-3, eta_min=1e-5, verbose=1)
]

In [ ]:
history = model.fit_generator(generator=training_generator,
                            validation_data=validation_generator,   
                           epochs=epochs, verbose=1,
                            callbacks=callbacks) 

In [ ]:
# list all data in history
import matplotlib.pyplot as plt
 
print(history.history.keys())

# summarize history for iou
plt.figure(figsize=(20,5))
plt.subplot(1,2,1)
plt.plot(history.history['my_iou_metric'])
plt.plot(history.history['val_my_iou_metric'])
plt.title('model IOU')
plt.ylabel('iou')
plt.xlabel('epoch')
plt.legend(['train', 'Validation'], loc='upper left')

# summarize history for loss
plt.subplot(1,2,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'Validation'], loc='upper left')

# Evaluation validation data

In [ ]:
params_val = {'img_size': img_size,
          'batch_size': 5,
          'n_channels': 3,
          'shuffle': False,
         }

# Generators
validation_generator = DataGenerator(X_val, masks, **params_val)

In [ ]:
AUGMENTATIONS_TEST_FLIPPED = A.Compose([
    A.HorizontalFlip(p=1),
],p=1)

params_val_flip = {'img_size': img_size,
          'batch_size': 5,
          'n_channels': 3,
          'shuffle': False,
        'augmentations':AUGMENTATIONS_TEST_FLIPPED,
         }

validation_generator_flipped = DataGenerator(X_val, masks, **params_val_flip)

In [ ]:
preds_valid_orig = predict_result(model,validation_generator,img_size)
preds_valid_flipped = predict_result(model,validation_generator_flipped,img_size)
preds_valid_flipped = np.array([np.fliplr(x) for x in preds_valid_flipped])
preds_valid = 0.5*preds_valid_orig + 0.5*preds_valid_flipped

In [ ]:
np.savez_compressed('process_data/val_pre/preds_valid_resnet50', array1= preds_valid)

In [ ]:
y_truth_val = label_generator(X_val, masks, len(preds_valid), img_size, 3)

np.savez_compressed('process_data/val_pre/y_truth_val', array1= y_truth_val)

In [ ]:
decompressed_array= np.load("process_data/val_pre/y_truth_val.npz")  
y_truth_val = decompressed_array['array1']

In [ ]:
## Scoring for last model
score = 0.0
mask_area = 0
best_th = 0

thresholds = np.arange(0.2, 0.9, 0.01) 
areas = [1024, 2048, 3072, 4096]
for threshold in tqdm(thresholds):
    for area in tqdm(areas):
        iou = iou_metric_batch_val(y_truth_val, np.int32(preds_valid > threshold), area)
        if iou > score:
            score = iou
            mask_area = area
            best_th = threshold
            print("Threshold {}\tMask area {}\tIoU {}".format(best_th, mask_area, score))
    print()

# Test Prediction

In [ ]:
test_file = 'stage2_siim_data/stage_2_images/*.dcm'
test_metadata_df = prepare_test(test_file, rle_file)

In [ ]:
test_data = get_test(3205, test_metadata_df, img_size=img_size, channels=3) #0, 1068, 2136, 3205
print(test_data.shape)

In [ ]:
resnet50_512_pred_test = get_prediction(model, test_data, batch_size=batch_size)

In [ ]:
np.savez_compressed('process_data/test_pre/resnet50_512_pred_test', array1= resnet50_512_pred_test)

In [ ]:
decompressed_array= np.load("process_data/test_pre/resnet50_512_pred_test.npz")  
resnet50_512_pred_test = decompressed_array['array1']

In [ ]:
rles = get_rles(preds_test, b_th = 0.73, r_th = 2048)

In [ ]:
test_fn = sorted(glob('stage2_siim_data/stage_2_images/*.dcm'))
test_IDs = [o.split('/')[-1][:-4] for o in test_fn]

In [ ]:
sub_df = pd.DataFrame({'ImageId': test_IDs, 'EncodedPixels': rles})
sub_df.loc[sub_df.EncodedPixels=='', 'EncodedPixels'] = '-1'
sub_df.head()

In [ ]:
sub_df.to_csv('model_submission/resnet50_submission.csv', index=False)

In [ ]:
sub_df['EncodedPixels'].value_counts(normalize=True) * 100